In [2]:
import pandas as  pd
import glob
import re
import math
methodology_text = """
<h2>Methodology</h2><p>The cheapest cities in each state were ranked based on its median home price and median asking rents for studio, one-, two-, and three-bedroom units. Prior to ranking, inputs were normalized, and weights were applied using a 1.25:1 ratio of asking rents to home prices.
Data on home prices are from the U.S. Census 2016-2020 American Community Survey 5-year estimates. Data on asking rents are from Rent.
Cities without data for one- or two-bedroom asking rents or a population of less than 10,000 were removed from this ranking. Any other missing values were zeroed and did not impact the final score.</p>"""
missed_content_urls = ['NH', "IN", 'FL', 'PA', "ME", 'HI', 'OR', "TN" ]

def get_embed_code(filepath):
    with open(filepath) as my_file:
        e_code = my_file.read()
    return e_code

def format_response(state, response, map_embed, table_embed, series_embed):
    myLines = response.split("\n")
    fin_lines = ""
    is_h1 = True
    count_all_h2s = len(re.findall("#{2} ", response))

    second_graph_pos = math.floor(count_all_h2s / 2)
    h2_count = 0

    second_graph = False
    my_header = ""
    for line in myLines:
        if re.match("^# ", line ):
            line = line.replace("# ", "")
            line = line.replace("#", "")
            my_header = line
            line = ""
        elif "## " in line:
            h2_count+=1 
            if h2_count == second_graph_pos:
                second_graph = True
            line = line.replace("## ", "<h2>")
            line += "</h2>"
        else: 
            if line != "":
                line = "<p>" + line + "</p>"
               
                if is_h1 == True:
                    is_h1 = False

                    line+= "<div>" + map_embed +"</div>"
                if second_graph == True:
                    print('second')
                    second_graph = False
                    if state == "VT" or state == "HI":
                        pass
                    else:
                        line+= "<div>" + series_embed + "</div>"
        fin_lines += line
    fin_lines += "<div>" + table_embed +"</div>"
    fin_lines += methodology_text
    return fin_lines, my_header
# Get gpt outputs.  format the headers, etc. Insert embed codes. 
output_file = pd.DataFrame(columns=['cheapest_state',  'header', 'html'])
for file in glob.glob("states/*/*.txt"):
    for stat_abv in missed_content_urls:
        if stat_abv in file:
            mygpt = pd.read_csv(file)
            pathing_to_graphs = "/".join(file.split("/")[:-1])
            time_series = get_embed_code(pathing_to_graphs + "/Timeseries/embed_code.txt")
            mapping = get_embed_code(pathing_to_graphs + "/Map/embed_code.txt")
            table = get_embed_code(pathing_to_graphs + "/Table/embed_code.txt")

            prod_ready_responses, header = format_response(file.split('/')[1], mygpt['response'].iloc[0], mapping, table, time_series)
            output_file.loc[len(output_file)] = [file.split('/')[1], header, prod_ready_responses]
output_file.to_csv("missed_results_simpel_script.csv", index=False)

second
second
second
second
second
second
second
